In [1]:
#  Подключаю необходимые библиотеки
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [3]:
df = pd.read_csv('winequality-red.csv', sep=';')

In [4]:
#  Все хорошо?
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
#  Классы сбалансированы?
df['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [6]:
#  разделяю предикоторы и отклики
X = df.iloc[:, :-1].values  
y = df.iloc[:, -1].values 

In [7]:
#  разделяю на обучающую и тестовую выборку

from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)  

In [8]:
# conda install py-xgboost
# conda install -c anaconda py-xgboost  -- появилась Teano
from xgboost import XGBClassifier

# from sklearn.metrics import classification_report

from sklearn import metrics

In [32]:
model = XGBClassifier(seed=13,
                      n_estimators=100,
                      max_depth=6,
                      learning_rate=0.1,
                      reg_alpha=0,
                      reg_lambda=1,
                      eval_metric='mlogloss')


model.fit(X_train, y_train)



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='mlogloss',
              gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=13,
              silent=None, subsample=1, verbosity=1)

In [33]:
# Определим тестируемые значения гиперпараметров
grid_param = {  
      'n_estimators': [100,  500, 1000],
      'max_depth': [3, 4, 5, 6],
      'learning_rate': [0.01, 0.1, 0.2],
      'reg_lambda': [0.001, 0.01, 0.1],
      'reg_alpha': [0.001, 0.01, 0.1]
  }

In [34]:
from sklearn.model_selection import GridSearchCV

In [35]:
# Проведем 5-fold кросс-валидацию
grid_CV = GridSearchCV(estimator=model,
                       param_grid=grid_param,
                       scoring='balanced_accuracy',
                       cv=5,
                       n_jobs=-1,
                       error_score='raise',
                       pre_dispatch='2*n_jobs',
                       refit=True
                      )

In [37]:
# Обучим на тренировочной выборке
grid_CV.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, eval_metric='mlogloss',
                                     gamma=0, learning_rate=0.1,
                                     max_delta_step=0, max_depth=6,
                                     min_child_weight=1, missing=None,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     objective='multi:softprob', random_state=0,
                                     reg_alp...bda=1,
                                     scale_pos_weight=1, seed=13, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5, 6],
               

In [38]:
# Выберем лучшую модель
best_model = grid_CV.best_estimator_
print('Лучшие значения гиперпараметров:\n', grid_CV.best_params_)

Лучшие значения гиперпараметров:
 {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 1000, 'reg_alpha': 0.001, 'reg_lambda': 0.001}


In [39]:
# Строим предсказания модели
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

print(metrics.classification_report(y_pred_test, y_test))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         2
           5       0.77      0.74      0.76       240
           6       0.61      0.65      0.63       204
           7       0.65      0.49      0.56        79
           8       0.00      0.00      0.00         1

    accuracy                           0.66       528
   macro avg       0.34      0.31      0.32       528
weighted avg       0.68      0.66      0.67       528



In [40]:
from sklearn.calibration import CalibratedClassifierCV

In [41]:
model_sigmoid = CalibratedClassifierCV(best_model, cv=2, method='sigmoid')
model_sigmoid.fit(X_train, y_train)

CalibratedClassifierCV(base_estimator=XGBClassifier(base_score=0.5,
                                                    booster='gbtree',
                                                    colsample_bylevel=1,
                                                    colsample_bynode=1,
                                                    colsample_bytree=1,
                                                    eval_metric='mlogloss',
                                                    gamma=0, learning_rate=0.01,
                                                    max_delta_step=0,
                                                    max_depth=6,
                                                    min_child_weight=1,
                                                    missing=None,
                                                    n_estimators=1000, n_jobs=1,
                                                    nthread=None,
                                                    objective='multi

In [42]:
y_pred_train = model_sigmoid.predict(X_train)
y_pred_test = model_sigmoid.predict(X_test)

print(metrics.classification_report(y_pred_test, y_test))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.79      0.73      0.76       251
           6       0.65      0.64      0.65       221
           7       0.52      0.55      0.53        56
           8       0.00      0.00      0.00         0

    accuracy                           0.67       528
   macro avg       0.33      0.32      0.32       528
weighted avg       0.71      0.67      0.69       528



C:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
# Строим предсказание модели
y_pred_test_probs = model_sigmoid.predict_proba(X_test)

# Оценим долю наблюдений в тестовой выборке, для которых есть класс, вероятность принадлежать которому больше 0.8
(y_pred_test_probs > 0.8).sum() / len(y_pred_test_probs)

0.0